In [1]:
import numpy as np
import os
import torch
import torchvision.models as models

#download the resnet18
resnet_sample = models.resnet18(pretrained = True)

use_cuda = torch.cuda.is_available()

In [2]:
print(resnet_sample)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Preprocess and load the data

In [3]:
from torchvision import datasets
import torchvision.transforms as transforms

train_transforms = transforms.Compose([transforms.Resize(size=258),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomRotation(10),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])

validTest_transforms = transforms.Compose([transforms.Resize(size=258),
                                           transforms.CenterCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                std=[0.229, 0.224, 0.225])])

train_path = datasets.ImageFolder('train',transform=train_transforms)
test_path = datasets.ImageFolder('test',transform=validTest_transforms)
valid_path = datasets.ImageFolder('val',transform=validTest_transforms)

loaders_scratch = {'train': torch.utils.data.DataLoader(train_path,
                                          batch_size=128,
                                          shuffle=True),

                   'test': torch.utils.data.DataLoader(test_path,
                                          batch_size=20,
                                          shuffle=True),

                   'val': torch.utils.data.DataLoader(valid_path,
                                          batch_size=20,
                                          shuffle=True)}

# Build resnet18 from scratch

In [4]:
import torch.nn as nn

class block(nn.Module):
    def __init__(self, in_channels, out_channels, in_channels_2, out_channels_2, identity_downsample=None, stride=1, bias=False):
        super(block, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace = True)
        self.conv2 = nn.Conv2d(in_channels_2, out_channels_2, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.identity_downsample = identity_downsample
        
    def forward(self,x):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x=self.relu(x)        
        x = self.conv2(x)
        x = self.bn2(x)
        
        #if self.identity_downsample is not None:
           # identity = self.identity_downsample(identity)
            
     

        return x

In [5]:
class ResNet(nn.Module):
    def __init__(self, block, image_channels, num_classes):
        super(ResNet, self).__init__()
        
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        
                
        self.layer1 = nn.Sequential(block(64,64,64,64),
                                    block(64,64,64,64))
        self.layer2 = nn.Sequential(block(64,128,128,128,
                                    identity_downsample = nn.Sequential(nn.Conv2d(64, 128, kernel_size=1,stride=2),nn.BatchNorm2d(128)),stride = 2),
                                    block(128,128,128,128))     
        self.layer3 = nn.Sequential(block(128,256,256,256,
                                    identity_downsample=nn.Sequential(nn.Conv2d(128, 256, kernel_size=1,stride=2),nn.BatchNorm2d(256)),stride = 2),
                                    block(256,256,256,256)) 
        self.layer4 = nn.Sequential(block(256,512,512,512,
                                    identity_downsample=nn.Sequential(nn.Conv2d(256, 512, kernel_size=1,stride=2),nn.BatchNorm2d(512)) ,stride = 2),
                                    block(512,512,512,512)) 
    
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x
    
    
resnet = ResNet(block, image_channels =3, num_classes=4)


if use_cuda:
    resnet.cuda()

In [6]:
print(resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel

# Specify Loss Function and Optimizer

In [8]:
import torch.optim as optim

resnet_criterion = nn.CrossEntropyLoss()

resnet_optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Train and validate the model

In [9]:
from PIL import ImageFile 
import time
ImageFile.LOAD_TRUNCATED_IMAGES = True
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        start_time = time.time()
        
        ###################
        # train the model #
        ###################
        model.train()

        for batch_idx, (data, target) in enumerate(loaders['train']):
            #print(f'Path: {path}\n')
            # move to GPU

            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer.zero_grad()

            output = model(data)

            loss = criterion(output, target)

            loss.backward()

            optimizer.step()

            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['val']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            
            loss = criterion(output,target)
            
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics 
        end_time = time.time()
        epoch_time = end_time - start_time
        print('Epoch: {} \tSec. spent in epoch {:.2f}\tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            epoch_time,
            train_loss,
            valid_loss
            ))
        
        ## save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.46} --> {:.6f}).  Saving model'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model


# train the model
resnet = train(10, loaders_scratch, resnet, resnet_optimizer, 
                      resnet_criterion, use_cuda, 'resnet.pt')

# load the model that got the best validation accuracy
resnet.load_state_dict(torch.load('resnet.pt'))

Epoch: 1 	Sec. spent in epoch 607.44	Training Loss: 0.455502 	Validation Loss: 0.284309
Validation loss decreased (inf --> 0.284309).  Saving model
Epoch: 2 	Sec. spent in epoch 606.18	Training Loss: 0.277682 	Validation Loss: 0.093300
Validation loss decreased (0.284309446811676025390625 --> 0.093300).  Saving model
Epoch: 3 	Sec. spent in epoch 614.16	Training Loss: 0.231080 	Validation Loss: 0.157183
Epoch: 4 	Sec. spent in epoch 612.08	Training Loss: 0.206280 	Validation Loss: 0.044317
Validation loss decreased (0.093299724161624908447265625 --> 0.044317).  Saving model
Epoch: 5 	Sec. spent in epoch 612.29	Training Loss: 0.189936 	Validation Loss: 0.126843
Epoch: 6 	Sec. spent in epoch 609.57	Training Loss: 0.177483 	Validation Loss: 0.059819
Epoch: 7 	Sec. spent in epoch 609.23	Training Loss: 0.167995 	Validation Loss: 0.041820
Validation loss decreased (0.04431696236133575439453125 --> 0.041820).  Saving model
Epoch: 8 	Sec. spent in epoch 616.76	Training Loss: 0.159680 	Validati

<All keys matched successfully>

In [10]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_scratch, resnet, resnet_criterion, use_cuda)

Test Loss: 0.023353


Test Accuracy: 99% (960/968)


# Train and test the pretrained model (Transfer Learning)

In [16]:
import torchvision.models as models
import torch.nn as nn

use_cuda = torch.cuda.is_available()

print(resnet_sample)

# Freeze parameters of the model to avoid brackpropagation
for param in resnet_sample.parameters():
    param.requires_grad = False

resnet_sample.fc = nn.Linear(512, 4)

# Rplace the original classifier
resnet_sample.classifier = resnet_sample.fc

if use_cuda:
    resnet_sample = resnet_sample.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.Adam(resnet_sample.parameters(),lr=0.001)

In [21]:
loaders_transfer = loaders_scratch
# train the model
resnet_sample =  train(10, loaders_transfer, resnet_sample, optimizer_transfer, criterion_transfer, use_cuda, 'resnet_sample.pt')

# load the best model
resnet_sample.load_state_dict(torch.load('resnet_sample.pt'))

Epoch: 1 	Sec. spent in epoch 518.06	Training Loss: 0.472878 	Validation Loss: 0.306696
Validation loss decreased (inf --> 0.306696).  Saving model
Epoch: 2 	Sec. spent in epoch 519.15	Training Loss: 0.467066 	Validation Loss: 0.316115
Epoch: 3 	Sec. spent in epoch 519.20	Training Loss: 0.470846 	Validation Loss: 0.282684
Validation loss decreased (0.306696414947509765625 --> 0.282684).  Saving model
Epoch: 4 	Sec. spent in epoch 517.88	Training Loss: 0.466681 	Validation Loss: 0.337931
Epoch: 5 	Sec. spent in epoch 512.71	Training Loss: 0.469961 	Validation Loss: 0.319104
Epoch: 6 	Sec. spent in epoch 499.44	Training Loss: 0.468309 	Validation Loss: 0.318729
Epoch: 7 	Sec. spent in epoch 495.42	Training Loss: 0.464372 	Validation Loss: 0.363941
Epoch: 8 	Sec. spent in epoch 500.86	Training Loss: 0.466772 	Validation Loss: 0.331088
Epoch: 9 	Sec. spent in epoch 472.68	Training Loss: 0.467911 	Validation Loss: 0.339625
Epoch: 10 	Sec. spent in epoch 472.78	Training Loss: 0.466868 	Valid

<All keys matched successfully>

In [22]:
test(loaders_transfer, resnet_sample, criterion_transfer, use_cuda)

Test Loss: 0.354219


Test Accuracy: 84% (815/968)
